#     Plan your trip with Kayak

###        Company's description 📇

Kayak is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by Booking Holdings which now holds:

   - Booking.com

   - Kayak
   
   - Priceline

   - Agoda

   - RentalCars

   - OpenTable

With over $300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe.


###     Project 🚧

The marketing team needs help on a new project. After doing some user research, 

the team discovered that 70% of their users who are planning a trip would like to have more information about the destination they are going to.

In addition, user research shows that people tend to be defiant about the information they are reading if they don't know the brand which produced the content.

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. 

The application should be based on real data about:
- Weather
- Hotels in the area

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time.

##        Goals 🎯
As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to:

- Scrape data from destinations
- Get weather data from each destination
- Get hotels' info about each destination
- Store all the information above in a data lake
- Extract, transform and load cleaned data from your datalake to a data warehouse

##  Introduction

In order to organise this projet, it was split into several parts; 

0. Definition
    - list of cities included in the projet (possibility to extend)
    - import libraries required
1. Gather location information from nominatim API https://nominatim.org/
    - latitude 
    - longitude, 
2. Gather weather information from openweathermap API https://openweathermap.org/appid
    - temperature
    - feels like
    - humidity
    - rain probability
    - expected 
3. Scrap Booking.com to gather the list of hotels in a defined city using scrapy
    - name
    - score
    - reviews
    - latitude
    - longitude
    - url
    - description
4. Creation of graphic display to present the information
    - Cities with best weather
    - Top 20 hotels in each city
5. Dataset and storage
    - Creation of a csv file, store into S3
    - Ceation of SQL Database using AWS RDS
    - Extraction of the data from S3 into a new DB

#   Project

##  Definition

In [2]:
# Importing necessary libraries for data manipulation, visualization, and interaction with web APIs.
import pandas as pd #To manipulate datasets

import json #Json data parsing
import requests #To interact with API

import subprocess #In order to launch the scrapy python file
import time  # Import the time module
import boto3

import plotly.graph_objects as go #Plot interactive figures
from plotly.offline import plot #Plot figures as html, more responsive

import warnings #To avoid warnings to take a lot of space
warnings.filterwarnings("ignore")

In [3]:
# Defining the list of cities of the project
list_cities = [
"Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"
]

In [4]:
#Keys
#Openweather API key
OpenweatherAPPID = # Your Openweather API key

##   Gather location information from nominatim API

In [5]:
""" This code looks for geolocation data in the nominatim API, read the obtained data in a json format.
    The geolocation data obtained are the osm_id (unique city id), latitude and longitude.
    The structure of the geolocation dataframe should look like this:
    | city | osm_id |  lat  |  lon  |
    | str  |   int  | float | float |
    """
#Gather location informationf from nominatim API
#using the requests library

list_geo = [] #Creation of an empty list to store the information recieved from API

#For loop to go through all the cities present in list_cities and obtain geolocation for each
for city in list_cities: 
    #API request witin a try-except block to handle errors
    try:
        print("Looking for city: {}".format(city))
        # Requests in the nominatim API situated in France, changing the query value as the city
        response = requests.get("https://nominatim.openstreetmap.org/search.php?q={}+France&format=jsonv2".format(city))
        # Read data as json
        results = response.json()
    except:
        # In case of an error
        print("Could not get information for city: {},".format(city), "error code :", response.status_code)
    else:
        #Storing the relevant data
        if results: #Checking results is not empty
            city_latitude = results[0]['lat']#Extract latitude
            city_longitude = results[0]['lon'] #Extract longitude
            city_osm_id = results[0]['osm_id']#Extract id

            #Stock the data into a list
            city_list = [city, city_osm_id, city_latitude, city_longitude]
            #Appending data from each city into list_geo
            list_geo.append(city_list)
        else:
            #In case there are no data
            print(f"No results found for city: {city}")

Looking for city: Mont Saint Michel
Looking for city: St Malo
Looking for city: Bayeux
Looking for city: Le Havre
Looking for city: Rouen
Looking for city: Paris
Looking for city: Amiens
Looking for city: Lille
Looking for city: Strasbourg
Looking for city: Chateau du Haut Koenigsbourg
Looking for city: Colmar
Looking for city: Eguisheim
Looking for city: Besancon
Looking for city: Dijon
Looking for city: Annecy
Looking for city: Grenoble
Looking for city: Lyon
Looking for city: Gorges du Verdon
Looking for city: Bormes les Mimosas
Looking for city: Cassis
Looking for city: Marseille
Looking for city: Aix en Provence
Looking for city: Avignon
Looking for city: Uzes
Looking for city: Nimes
Looking for city: Aigues Mortes
Looking for city: Saintes Maries de la mer
Looking for city: Collioure
Looking for city: Carcassonne
Looking for city: Ariege
Looking for city: Toulouse
Looking for city: Montauban
Looking for city: Biarritz
Looking for city: Bayonne
Looking for city: La Rochelle


In [6]:
#Creation of a dataframe, using Pandas library, from the data stocked in list_geo, with defined columns
geo_df = pd.DataFrame(list_geo, columns=('city', 'osm_id', 'lat', 'lon'))
display(geo_df)

,city,osm_id,lat,lon
0,Mont Saint Michel,211285890,48.6359541,-1.511459954959514
1,St Malo,905534,48.649518,-2.0260409
2,Bayeux,145776,49.2764624,-0.7024738
3,Le Havre,104492,49.4938975,0.1079732
4,Rouen,75628,49.4404591,1.0939658
5,Paris,7444,48.8588897,2.3200410217200766
6,Amiens,114347,49.8941708,2.2956951
7,Lille,58404,50.6365654,3.0635282
8,Strasbourg,71033,48.584614,7.7507127
9,Chateau du Haut Koenigsbourg,61044809,48.249489800000006,7.34429620253195


##   Gather weather information from openweathermap API

In [7]:
#Preprocessing of the data in the dataframe
#Changing spaces by & for all cases
geo_df["city"] = geo_df["city"].str.replace(" ", "&")
#In some edge case, custom changes
geo_df["city"][geo_df["city"] == "La&Rochelle"] = "La Rochelle"
geo_df["city"][geo_df["city"] == "St&Malo"] = "Saint-Malo"
geo_df["city"][geo_df["city"] == "Le&Havre"] = "Le Havre"


In [48]:
""" This code looks for weather data in the openweather API, read the obtained data in a json format.
    The weather data is obtained for the 5 next days, with an interval of 3 hours between prediction points.
    For a defined city, the weather will contain 24h/3h = 8 data points per day, and 40 for the full 5 days.
    The structure of the final dictionaty city_weather should look like this:
    city weather =  {'city_name': 
                            {'id': city_id,
                            {'weather': [
                                        {'day': day1, ...},
                                        {'day': day2, ...},
                                        {'day': day3, ...}...
                                    ]
                            }
                    }"""



#Gather weather information from openweathermap API
#Creation of an empty dictionary to stock the weather information keyed by city names
city_weather = {}
#Defining the list of cities as the list present in the dataframe column 'city'
cities = geo_df["city"]

#For loop to go through all the cities present in cities and obtain weather information for each
for city in cities:
    print(f"Looking for weather in city: {city}")
    # GET Requests in the openweather API 5-day Forecast situated in France 'fr', with metric units, changing the query value as the city
    response = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?q={city},fr&units=metric&appid={OpenweatherAPPID}")
    
    # Retrieve the OSM ID from the geo_df DataFrame for the corresponding city
    id = geo_df['osm_id'][geo_df['city'] == city].values[0]
    
    # Check if the request was successful
    if response.status_code == 200:
        response_data = response.json() #Read in json format
        temp_weather = []  # Initialize the list to hold weather data for the current city
        
        #Stock osm id in a dictionary for future use
        temp_id = {
                    'osm_id': id
                }
        #Stock the id dictionary in the city_weather dictionary
        city_weather[city] = temp_id
        # Extract weather data
        for weather_data in response_data['list']:
            #try-except block to handle errors
            try:
                date = weather_data['dt_txt'].split(' ') #Extract date formated as '##:##:## 00:00:00' and split
                day = date[0] #Day
                hour = date[1] #Hour
                temp = weather_data['main']['temp'] #Extract temperature
                feels_like = weather_data['main']['feels_like'] #Extract temperature felt
                humidity = weather_data['main']['humidity'] #Extract humidity
                weather_main = weather_data['weather'][0]['main'] #Extract weather category such as 'Rain'
                weather_desc = weather_data['weather'][0]['description'] #Extract weather description subcategories such as 'light rain'
                # Use get to avoid KeyError if 'pop' does not exist
                rain_pop = weather_data.get('pop', 0) #Extract rain probability
                rain_mmh = weather_data.get('rain', {}).get('3h', 0)  #Extract rain mm/hour 
            except Exception as e:
                print(f"Could not find the information: {e}")
            else:
                #Stock extracted data into a temporary dictionary
                temp_dict = {
                    'day': day,
                    'hour': hour,
                    'temp': temp,
                    'feels_like': feels_like,
                    'humidity': humidity,
                    'weather_main': weather_main,
                    'weather_desc': weather_desc,
                    'rain_pop': rain_pop,
                    'rain_mmh': rain_mmh
                }
                #Add temporary dictionary into the initiated empty list
                temp_weather.append(temp_dict)
                
        #Add the weather data list to the city_weather dictionary with the key 'weather'
        city_weather[city]['weather'] = temp_weather# Add the weather data for the current city to the main dictionary
    else:
        print(f"Error fetching weather data for {city}: {response.status_code}")

Looking for weather in city: Mont&Saint&Michel
Looking for weather in city: Saint-Malo
Looking for weather in city: Bayeux
Looking for weather in city: Le Havre
Looking for weather in city: Rouen
Looking for weather in city: Paris
Looking for weather in city: Amiens
Looking for weather in city: Lille
Looking for weather in city: Strasbourg
Looking for weather in city: Chateau&du&Haut&Koenigsbourg
Looking for weather in city: Colmar
Looking for weather in city: Eguisheim
Looking for weather in city: Besancon
Looking for weather in city: Dijon
Looking for weather in city: Annecy
Looking for weather in city: Grenoble
Looking for weather in city: Lyon
Looking for weather in city: Gorges&du&Verdon
Looking for weather in city: Bormes&les&Mimosas
Looking for weather in city: Cassis
Looking for weather in city: Marseille
Looking for weather in city: Aix&en&Provence
Looking for weather in city: Avignon
Looking for weather in city: Uzes
Looking for weather in city: Nimes
Looking for weather in c

##   Scrap Booking.com to gather the list of hotels in a defined city using scrapy

In [8]:
""" This code manages the use of the scrapy spider in order to obtain information from booking.com.
    The file, launched with subprocess, is 'scrapy_booking/scrapy_final_csv.py', 
    and requires the name of the output file and two inputs formdata;
    the formdata : containing information of the city used in the query,
    the value : an offset used to change page (25 hotels per page), 
    and an order : bayesian_review_score which rank the hotel based on the score.
    
    A for loop is used to launch the scrap process for each city defined
    in the list of cities, and each page defined in the input page.
    
    The rate of success should be around 60% due to various reasons, the failed processes
    are stocked into a file : fails.jsonl. In a second step, the content of the fails.json1
    file is stock into an empty list called fail_list, this list is then used as the input
    to launch the scrap process only for the failed cases.
    
    This step may need to be done multiple time until all the files are complete.
    
    The output of the scrap process, gives a csv file per city, containing all the pages.
    The structure of the csv file for a city should look like this:
    | hotel  | score |  reviews  |  lat  |   lon |   url  | description |
    | hotel 1| float |     int   | float | float | string |    string   |
    ....
    """
                    
filename_fail = "fails.jsonl" #Naming the file containing the failed processes
fail_list = [] #Creation of an empty list to contain the failed
cities = geo_df['city']
# Delay in seconds
delay = 0.5  # Delay between processes
page = 2 #Number of pages per search

# open the fail file in write mode to truncate it
with open(filename_fail, 'w') as file:
    pass

"""For loop starting at 0 for an offset = 0 which corresponds to the first page
The interval is set at 25 corresponding to the number of elements in one page
The max value is 0 for page = 1, 26 for page = 2, 51 for page = 3 etc.
"""
for n in range(0, ((page-1)*25)+1, 25):
    page_n = n // 25 +1 #page number, for n = 0 : page_n == 1 etc.
    #For loop for city in the list cities
    for city in cities:    
        #formdata input to the scrap process
        formdata = {"ss": city, "offset": str(n), "order": "bayesian_review_score"}
        #Set filename of output file
        filename = f'{city}.csv'
        
        print(city, "page :", page_n)
        #Subprocess launching the spider, with input formdata and name of the outpput file
        subprocess.run(['python', 'scrapy_final_csv.py', '-f', json.dumps(formdata), '-o', filename])
        
        # Wait for 'delay' seconds before starting the next process
        time.sleep(delay)
        
k = 1 #Initiating a factor k in order to process until all the data is obtained with a while loop
while k != 0:
    fail_list = [] #Resetting fail_list at the start of each loop
    # Read the content and load into fail_list
    with open(filename_fail, 'r') as file:
        for line in file:
            # Parse the JSON line
            fail_list.append(json.loads(line))

    # open the file again in write mode to truncate it
    with open(filename_fail, 'w') as file:
        pass

    k = len(fail_list) #Gives the value to k, if no data left to scrap k = 0
    print(f'The number of failed elements is: {k}')
    if k != 0:
        print('Relaunching the scraping...sorry for the delay')
    else:
        pass
    #Relaunch the scraping based on the fail_list
    for i in range(len(fail_list)): 
            formdata = fail_list[i]
            page_n = int(formdata['offset']) // 25 +1
            city = fail_list[i]['ss']
            filename = f'{city}.csv'
            print(formdata['ss'], 'page :', page_n)
            subprocess.run(['python', 'scrapy_final_csv.py', '-f', json.dumps(formdata), '-o', filename])

            # Wait for 'delay' seconds before starting the next process
            time.sleep(delay)

Mont&Saint&Michel page : 1
Saint-Malo page : 1
Bayeux page : 1
Le Havre page : 1
Rouen page : 1
Paris page : 1
Amiens page : 1
Lille page : 1
Strasbourg page : 1
Chateau&du&Haut&Koenigsbourg page : 1
Colmar page : 1
Eguisheim page : 1
Besancon page : 1
Dijon page : 1
Annecy page : 1
Grenoble page : 1
Lyon page : 1
Gorges&du&Verdon page : 1
Bormes&les&Mimosas page : 1
Cassis page : 1
Marseille page : 1
Aix&en&Provence page : 1
Avignon page : 1
Uzes page : 1
Nimes page : 1
Aigues&Mortes page : 1
Saintes&Maries&de&la&mer page : 1
Collioure page : 1
Carcassonne page : 1
Ariege page : 1
Toulouse page : 1
Montauban page : 1
Biarritz page : 1
Bayonne page : 1
La Rochelle page : 1
Mont&Saint&Michel page : 2
Saint-Malo page : 2
Bayeux page : 2
Le Havre page : 2
Rouen page : 2
Paris page : 2
Amiens page : 2
Lille page : 2
Strasbourg page : 2
Chateau&du&Haut&Koenigsbourg page : 2
Colmar page : 2
Eguisheim page : 2
Besancon page : 2
Dijon page : 2
Annecy page : 2
Grenoble page : 2
Lyon page : 2
Go

In [50]:
"""This process here, read all the csv files created before, clean the data,
and stock it in a dictionary hotel_dict.
The structure of the file should be as such:
    hotel_dict = {  'city1' : 
                            | hotel_name  | score |  reviews  |  latitude  |   longitude |   url  | description | rank | rank_inv|
                            | hotel 1     | float |     int   |      float |       float | string |    string   |  int |  int    |,
                    'city2' :
                            | hotel_name  | score |  reviews  |  latitude  |   longitude |   url  | description | rank | rank_inv|
                            | hotel 1     | float |     int   |      float |       float | string |    string   |  int |  int    |,
    }
"""
#Number of review minimum
rev_num = 50
#Creation of an empty dictionary to stock the hotel data
hotel_dict = {}
for city in cities:
    #filename corresponding to the csv file
    filename = f'C:/Users/Kito/Youssef_Github/Projet_data/Projet/ETL_Kayak/source/{city}.csv'
    #Retreiving the id of the city
    id = city_weather[city]['osm_id']
    # Read the CSV file using pandas
    data = pd.read_csv(filename)
    data = data.dropna() #Drop NaN rows
    data = data[data['hotel_name'] != 'hotel_name'] #Drop a specific line
    data['hotel_reviews'] = data['hotel_reviews'].str.extract('(\d+\.?\d*)').astype(float) #Extract the number of reviews, and set type as float
    data['hotel_score'] = data['hotel_score'].str.extract('(\d+\.?\d*)').astype(float) #Extract score in case there are string as well, and set type as float
    data['hotel_description'] = data['hotel_description'].astype(str) #Set type as string
    
    #select only the data where the reviews number is higher than a number 'rev_num'
    data = data[data['hotel_reviews'] > rev_num]
    
    data['hotel_rank'] = data['hotel_score'].rank(ascending = False).astype(int) #Create rank column as type int based on the score
    data['hotel_rank_inv'] = data['hotel_score'].rank(ascending = True).astype(int) #Create the inversed rank as type int
    data['osm_id'] = id.astype(int) #Create a new column with id
    
    #Set type float to the columns 'lat', 'lon'
    cat_type = ['hotel_lat', 'hotel_lon']
    for cat in cat_type:
        data[cat] = data[cat].astype(float)
        
    #Reorder the dataset based on score, and reset the index
    data = data.sort_values(by='hotel_score', ascending=False).reset_index().drop(columns='index')
    
    #Put the data in the dictionary with the city as key, and select only the data where the reviews number is higher than 50
    hotel_dict[f'{city}'] = data
    


In [51]:
#Preprocessing of the data in the dataframe
#Changing spaces by & for all cases
geo_df["city"] = geo_df["city"].str.replace(" ", "&")
#In some edge case, custom changes
geo_df["city"][geo_df["city"] == "La&Rochelle"] = "La Rochelle"
geo_df["city"][geo_df["city"] == "St&Malo"] = "Saint-Malo"
geo_df["city"][geo_df["city"] == "Le&Havre"] = "Le Havre"

In [126]:
display(hotel_dict['Amiens']) #Example for Amiens

,hotel_name,hotel_score,hotel_reviews,hotel_lat,hotel_lon,hotel_url,hotel_description,hotel_rank,hotel_rank_inv,osm_id
0,LE CANGE,9.5,62.0,49.896384,2.306378,https://www.booking.com/hotel/fr/le-cange.en-g...,"LE CANGE, a property with a garden and a terra...",1,27,114347
1,Urbs Amiens Sud,9.5,85.0,49.876463,2.268443,https://www.booking.com/hotel/fr/urbs-app-b-am...,"Located in Amiens in the Picardy region, with ...",1,27,114347
2,Hyper centre idéal gare Amiens,9.4,67.0,49.892279,2.305065,https://www.booking.com/hotel/fr/hyper-centre-...,"Located 400 metres from Amiens Train Station, ...",4,25,114347
3,"""Au jardin sur l'eau""",9.4,365.0,49.886683,2.335212,https://www.booking.com/hotel/fr/au-jardin-sur...,"Situated in Amiens, ""Au jardin sur l'eau"" feat...",4,25,114347
4,un Nid en Ville-a Nest in Town -AMIENS-classé ...,9.4,93.0,49.891101,2.293930,https://www.booking.com/hotel/fr/un-nid-en-vil...,Situated in Amiens and only 1.3 km from Amiens...,4,25,114347
5,Appartement 2 personnes : LES CANAUX,9.3,55.0,49.896102,2.310790,https://www.booking.com/hotel/fr/appartement-2...,Appartement 2 personnes: LES CANAUX in Amiens ...,7,21,114347
6,Le Cottage des Hortillonnages,9.3,61.0,49.897552,2.316359,https://www.booking.com/hotel/fr/le-cottage-de...,"Located in Amiens, 1.5 km from Amiens Train St...",7,21,114347
7,L’ÉMERAUDE AMIENS,9.3,74.0,49.870006,2.308068,https://www.booking.com/hotel/fr/emeraude-amie...,"Boasting a spa bath, L’ÉMERAUDE AMIENS is set ...",7,21,114347
8,La vallée des Hortillonages,9.3,203.0,49.891860,2.318600,https://www.booking.com/hotel/fr/la-vallee-des...,"Offering garden views, La vallée des Hortillon...",7,21,114347
9,L'Escale 80,9.2,55.0,49.896419,2.306370,https://www.booking.com/hotel/fr/l-39-escale-8...,"Located in Amiens, L’Escale 80 is 600 metres f...",12,17,114347


##   Creation of graphic display to present the information

In [52]:
"""Now, we have obtained three datasets, geolocation,  weather and one for the hotels.
In order to present data, we chose to look at the average weather of the next five days 
during the day. This mean that for each city, the average weather will be calculated, 
and  we will merge geolocation and weather dataset.
"""
avg_weather = [] #Creation of an empty list to store the ave_weather

for city in cities:
    
    #Retreiving the id of the city
    id = city_weather[city]['osm_id']
    #Creation of a temporary dataframe, containing the value of the 'weather' key of the city
    temp_hour_df = pd.DataFrame(city_weather[city]['weather'])
    #Filter the data, including only day data between 9h and 21h
    temp_hour_df['hour'] = temp_hour_df['hour'][temp_hour_df['hour'].between('09:00:00','21:00:00')]
    temp_hour_df = temp_hour_df.dropna(subset=['hour']) #Drop NaN values
    #Calculation of mean
    avg_temp = temp_hour_df['temp'].mean().round(2) #Average temperature
    avg_fl = temp_hour_df['feels_like'].mean().round(2) #Average perceived temperature
    avg_humidity = temp_hour_df['humidity'].mean().round(2) #Average humidity
    avg_pop = temp_hour_df['rain_pop'].mean().round(2) #Average probability of rain
    avg_rain = temp_hour_df['rain_mmh'].mean().round(2) #Average quantity per hour of rain
    common_weather = temp_hour_df['weather_main'].mode()[0] #Most common weather using mode, giving the most frequent string 
    
    #Storing the data in a temporary dictionary
    avg_temp_weather = {
        'city': city,
        'osm_id': id,
        'temperature': avg_temp,
        'feels_like': avg_fl,
        'humidity': avg_humidity,
        'rain_pop': avg_pop,
        'rain_mmh': avg_rain,
        'expected_weather': common_weather
    }
    #Add the stored data in the avg_weather list
    avg_weather.append(avg_temp_weather)

#Creation of a dataframe from the list avg_weather
avg_weather_df = pd.DataFrame(avg_weather)

#Merging the two dataframes geo_df and avg_weather_df based on the osm_id of the city
geo_weather = geo_df[['city', 'osm_id', 'lat', 'lon']].merge(avg_weather_df[['osm_id','temperature', 'feels_like', 'humidity', 'rain_pop', 'rain_mmh', 'expected_weather']], on= 'osm_id')

#Changing the type of columns into float
cat_type = ['lat', 'lon', 'temperature', 'feels_like', 'humidity', 'rain_pop', 'rain_mmh']
for cat in cat_type:
    geo_weather[cat] = geo_weather[cat].astype(float)

#Creation of a rank_temp column based on the percieved temperature column
geo_weather['rank_temp'] = geo_weather['feels_like'].rank(ascending= False)

#Creation of rank_rain_inv column, based on the inverse of the probability of rain column. Which means rank 1 is least probable to have rain
geo_weather['rank_rain_inv'] = geo_weather['rain_pop'].rank(ascending= True)

#Creation of an rank_avg consisting of the average of the two just created columns, 
#consisting of showing the city with better temperature and least change to rain
geo_weather['rank_avg'] = (geo_weather['rank_rain_inv'] + geo_weather['rank_temp']) / 2

#Sort the dataframe based on new rank_avg columns
geo_weather = geo_weather.sort_values(by='rank_avg', ascending=True).reset_index().drop(columns='index')

In [12]:
#Top 5 city correspond then to the first 5
geo_weather_top5 = geo_weather.iloc[0:5]
geo_weather_top5

,city,osm_id,lat,lon,temperature,feels_like,humidity,rain_pop,rain_mmh,expected_weather,rank_temp,rank_rain_inv,rank_avg
0,Aigues&Mortes,272287,43.566152,4.191540,16.38,15.37,49.72,0.00,0.00,Clouds,2.0,1.0,1.50
1,Cassis,76425,43.214036,5.539632,14.06,13.23,65.52,0.10,0.76,Clear,4.0,3.5,3.75
2,Chateau&du&Haut&Koenigsbourg,61044809,48.249490,7.344296,27.24,29.04,70.64,0.25,0.16,Clouds,1.0,8.0,4.50
3,Collioure,18409,42.525050,3.083155,14.41,13.56,63.36,0.17,0.11,Clouds,3.0,7.0,5.00
4,Nimes,378685,43.837425,4.360069,12.03,11.11,68.56,0.11,0.34,Clouds,7.0,5.0,6.00


In [62]:
"""here, we propose to plot a figure showing first all the cities with a colorscale based on the average temperature of the next 5 days.
And the possibility to show with a button, the top 5 cities.
"""

# Create a figure
fig = go.Figure()

# Customization for color scale appearance
color_scale_config = dict(
    colorscale='Rainbow',  # Style
    cmin=geo_weather['temperature'].min(),  # minimum value of color scale
    cmax=geo_weather['temperature'].max(),  # maximum value of color scale
    showscale=True,  # to show the color scale
    colorbar=dict(
    title='Temperature (°C)',  # Color scale legend title
    ticktext=['°C'],  # Text labels for the tickvals
    )
)

# Trace for all cities
fig.add_trace(
    go.Scattermapbox(
        lat=geo_weather['lat'],
        lon=geo_weather['lon'],
        mode='markers+text',  #markers and label
        marker=go.scattermapbox.Marker(
            size=12,  # Size
            color=geo_weather['temperature'], #Color parameter
            opacity=0.7,  # Adjust opacity to make markers more or less prominent
            **color_scale_config  # Add color scale configuration here
        ),
        #text=geo_weather['city'], #Label
        name='All Cities' #Trace name
    )
)

# Trace for top 5 cities
fig.add_trace(
    go.Scattermapbox(
        lat=geo_weather_top5['lat'],
        lon=geo_weather_top5['lon'],
        mode='markers+text',  # Change this to 'markers' if you don't want text labels
        marker=go.scattermapbox.Marker(
            size=12,  # Adjust size accordingly
            color=geo_weather_top5['temperature'], #Color parameter
            opacity=0.7,  # Adjust opacity to make markers more or less prominent
            **color_scale_config  # Make sure top5 uses the same scale
        ),
        text=geo_weather_top5['city'],#Label
        name='Top 5 Cities', #Trace name
        visible=False #Non visible at first
    )
)

# Update the layout with mapbox details
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=4,
    mapbox_center={"lat": geo_weather['lat'].mean(), "lon": geo_weather['lon'].mean()}, #Centered on the average of geolocations coordinates
    width=1200,
    height=900,
    updatemenus=[
        go.layout.Updatemenu(
            type="buttons",
            direction="right",
            active=0, #First trace active
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top",
            buttons=[
                dict(label="All Cities",
                     method="update",
                     args=[{"visible": [True, False]}]),  # Show all cities, hide top 5
                dict(label="Top 5 Cities",
                     method="update",
                     args=[{"visible": [False, True]}]),  # Hide all cities, show top 5
            ],
        ),
    ],
    showlegend=False
)

plot(fig)

'temp-plot.html'

In [63]:
"""here, we propose to plot a figure showing the top 20 hotels per city, with a button to chose a city, the city order is based on weather rank.
Then on selection of the city, it display the top 20 hotels, with a colorscale depending on score and size depending on rank, the bigger the better.
"""

# Create a function that will add line breaks every n words
def add_linebreaks(s, n=10): # default is 10 words before a line break
    words = s.split() #Split all the words into a list
    if len(words) < n: # If the total number of words is less than 'n', return the original string.
        return s
    #if the list is longer than n, then split it when n is reached and add a linebreaker
    return '<br>'.join([' '.join(words[i:i+n]) for i in range(0, len(words), n)])

# Create a figure
fig = go.Figure()

#The figure will first be centered around the best city
Top_city = geo_weather['city'][0]
Top_city_hotel = hotel_dict[Top_city]

#Create empty list to store the buttons
buttons = []

color_scale_config = 'Rainbow'  # Just specify the color scale for marker

# Define the color bar
colorbar_dict = dict(
    title='Score',
    titleside='right',
    thicknessmode='pixels',  # Specify the mode for thickness, it could be 'fraction' or 'pixels'
    thickness=15,  # Specify thickness size which is the width of the color bar
    lenmode='pixels',  # Specify the mode for length, it could be 'fraction' or 'pixels'
    len=300,# Specify length size which is the height of the color bar
)

# For loop, based on index value and the list of cities
for i, city in enumerate(geo_weather['city']):
    # Trace for top 20 cities
    temp_hotel_df = hotel_dict[city].iloc[0:19]
    
    # Calculate mean values of latitude and longitude
    mean_lat = geo_weather.loc[geo_weather['city'] == city, 'lat'].mean()
    mean_lon = geo_weather.loc[geo_weather['city'] == city, 'lon'].mean()
    
    
    visible_traces = [False] * len(geo_weather['city'])  # All traces are initially invisible
    visible_traces[i] = True  # Only the current trace is visible

    
    # Apply the function to create a new column for hover text
    temp_hotel_df['hover_text'] = temp_hotel_df['hotel_description'].apply(lambda x: add_linebreaks(x, 10)) # Adjust n as needed
    
    fig.add_trace(
        go.Scattermapbox(
            lat=temp_hotel_df['hotel_lat'],
            lon=temp_hotel_df['hotel_lon'],
            mode='markers+text', 
            marker=go.scattermapbox.Marker(
                size=temp_hotel_df['hotel_rank_inv'],  # Adjust size accordingly
                color=temp_hotel_df['hotel_score'], #Colorscale
                colorscale=color_scale_config,  # Apply color scale
                cmin=temp_hotel_df['hotel_score'].min(),  # Minimum color value
                cmax=temp_hotel_df['hotel_score'].max(),  # Maximum color value
                colorbar = dict(
                    title='Score',
                    thicknessmode='pixels',  # Specify the mode for thickness, it could be 'fraction' or 'pixels'
                    thickness=15,  # Specify thickness size which is the width of the color bar
                    lenmode='pixels',  # Specify the mode for length, it could be 'fraction' or 'pixels'
                    len=300,# Specify length size which is the height of the color bar
                )
                
            ),
            text=temp_hotel_df['hotel_name'], 
            hoverinfo='text',
            showlegend=False,
            hovertext=temp_hotel_df['hover_text'],
            name=f'{city} Hotels',
            visible=False  # Initially all traces are invisible
        )
    )
    button = dict(
        label=city,
        method="update", #Uptade the view and markers
        args=[{"visible": visible_traces}, #Applying visible traces
            {"mapbox.center.lat": mean_lat, "mapbox.center.lon": mean_lon, "mapbox.zoom": 12}],  # Update the map center and zoom
    )
    
    buttons.append(button)


# Update the layout with mapbox details
fig.update_layout(
    mapbox={
        'style': "open-street-map",
        'zoom': 14,  # or any preferred zoom level
        'center': go.layout.mapbox.Center(lat=Top_city_hotel['hotel_lat'].mean(), lon=Top_city_hotel['hotel_lon'].mean())  # initial map center
    },
    width=1200,
    height=900,
    updatemenus=[{
        "buttons": buttons,
        "direction": "down",
        "pad": {"r": 10, "t": 10},
        "showactive": True,
        'x': 0.05,
        "xanchor": "left",
        'y': 0.05,
        "yanchor": "top"
    }],
    # Set the size of the map or remove if not needed
    autosize=True,
    coloraxis=dict(colorscale=color_scale_config, colorbar_title='Score'),  # Adjust this title
    showlegend=True,
)


plot(fig)

'temp-plot.html'

##  Dataset and storage

In [ ]:
hotel_df_list = []
# Iterate over the hotel_dict items
for city, hotels in hotel_dict.items():
    # For each city, create a DataFrame
    city_hotel_df = pd.DataFrame(hotels)
    # Add a 'city' column with the cleaned city name
    city_hotel_df['city'] = city.replace('&', ' ')  # Replace '&' with spaces if necessary
    # Append the DataFrame to the list
    hotel_df_list.append(city_hotel_df)
    
# Concatenate all DataFrames in the list into one DataFrame
combined_hotel_df = pd.concat(hotel_df_list, ignore_index=True)

In [56]:
weather_df_list = []
for city, weather in city_weather.items():
    # For each city, create a DataFrame
    city_weather_df = pd.DataFrame(weather)
    
    # Check if the 'weather' column is a stringified JSON and convert it if necessary
    if isinstance(city_weather_df['weather'].iloc[0], str):
        city_weather_df['weather'] = city_weather_df['weather'].apply(json.loads)
    
    # Use json_normalize to convert dict in 'weather' to DataFrame
    temp_weather_df = pd.json_normalize(city_weather_df['weather'])

    # Concatenate the new 'weather' DataFrame with the original 'df' and exclude the old 'weather' column
    city_weather_df = pd.concat([city_weather_df.drop('weather', axis=1), temp_weather_df], axis=1)
    # Add a 'city' column with the cleaned city name
    city_weather_df['city'] = city.replace('&', ' ')  # Replace '&' with spaces if necessary
    # Append the DataFrame to the list
    weather_df_list.append(city_weather_df)
    
# Concatenate all DataFrames in the list into one DataFrame
combined_weather_df = pd.concat(weather_df_list, ignore_index=True)

#Merging the two dataframes geo_df and avg_weather_df based on the osm_id of the city
combined_geo_weather_df = geo_df[['city', 'osm_id', 'lat', 'lon']].merge(combined_weather_df[['osm_id','day','hour','temp','feels_like','humidity','weather_main','weather_desc','rain_pop','rain_mmh']], on= 'osm_id')

In [58]:
combined_geo_weather_df

,city,osm_id,lat,lon,day,hour,temp,feels_like,humidity,weather_main,weather_desc,rain_pop,rain_mmh
0,Mont&Saint&Michel,211285890,48.6359541,-1.511459954959514,2023-11-06,12:00:00,12.17,11.43,76,Rain,light rain,0.83,0.45
1,Mont&Saint&Michel,211285890,48.6359541,-1.511459954959514,2023-11-06,15:00:00,12.09,11.34,76,Rain,light rain,0.89,0.56
2,Mont&Saint&Michel,211285890,48.6359541,-1.511459954959514,2023-11-06,18:00:00,9.28,6.91,87,Rain,light rain,0.87,0.81
3,Mont&Saint&Michel,211285890,48.6359541,-1.511459954959514,2023-11-06,21:00:00,8.82,6.35,89,Rain,light rain,0.73,1.50
4,Mont&Saint&Michel,211285890,48.6359541,-1.511459954959514,2023-11-07,00:00:00,7.96,5.43,87,Rain,light rain,0.69,0.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,La Rochelle,117858,46.1591126,-1.1520434,2023-11-10,21:00:00,12.56,11.73,71,Rain,light rain,0.20,0.10
1396,La Rochelle,117858,46.1591126,-1.1520434,2023-11-11,00:00:00,12.36,11.56,73,Rain,light rain,0.20,0.21
1397,La Rochelle,117858,46.1591126,-1.1520434,2023-11-11,03:00:00,12.00,11.19,74,Rain,light rain,0.28,0.24
1398,La Rochelle,117858,46.1591126,-1.1520434,2023-11-11,06:00:00,11.90,11.08,74,Rain,light rain,0.20,0.14


In [59]:
#   Creation of a database_weather_df, merging the combined_geo_weather_df and combined_hotel_df
database_weather_df = combined_geo_weather_df[['city',
                        'osm_id',
                        'lat',
                        'lon',
                        'day',
                        'hour',
                        'temp',
                        'feels_like',
                        'humidity',
                        'weather_main',
                        'weather_desc',
                        'rain_pop',
                        'rain_mmh',
                        ]].merge(combined_hotel_df[['hotel_name',
                                                            'hotel_score',
                                                            'hotel_reviews',
                                                            'hotel_lat',
                                                            'hotel_lon',
                                                            'hotel_url',
                                                            'hotel_description',
                                                            'hotel_rank',
                                                            'hotel_rank_inv',
                                                            'osm_id']], on= 'osm_id')

In [60]:
#   Show the first 5 rows of the database_weather_df
database_weather_df.head()

,city,osm_id,lat,lon,day,hour,temp,feels_like,humidity,weather_main,...,rain_mmh,hotel_name,hotel_score,hotel_reviews,hotel_lat,hotel_lon,hotel_url,hotel_description,hotel_rank,hotel_rank_inv
0,Mont&Saint&Michel,211285890,48.6359541,-1.511459954959514,2023-11-06,12:00:00,12.17,11.43,76,Rain,...,0.45,L'Aurore de la Baie,9.7,583.0,48.609807,-1.452512,https://www.booking.com/hotel/fr/l-39-aurore-d...,"L'Aurore de la Baie features garden views, fre...",1,24
1,Mont&Saint&Michel,211285890,48.6359541,-1.511459954959514,2023-11-06,12:00:00,12.17,11.43,76,Rain,...,0.45,Le Domaine de Belleville,9.6,159.0,48.714573,-1.485254,https://www.booking.com/hotel/fr/le-domaine-de...,"Located in Dragey-Ronthon, Le Domaine de Belle...",2,23
2,Mont&Saint&Michel,211285890,48.6359541,-1.511459954959514,2023-11-06,12:00:00,12.17,11.43,76,Rain,...,0.45,Le grenier du jardin,9.5,602.0,48.553797,-1.503635,https://www.booking.com/hotel/fr/le-grenier-du...,"Located in Pontorson, 10 km from Mont Saint Mi...",5,20
3,Mont&Saint&Michel,211285890,48.6359541,-1.511459954959514,2023-11-06,12:00:00,12.17,11.43,76,Rain,...,0.45,Villa Mons,9.5,361.0,48.554225,-1.509382,https://www.booking.com/hotel/fr/villa-mons.en...,"Located in the centre of Pontorson, Villa Mons...",5,20
4,Mont&Saint&Michel,211285890,48.6359541,-1.511459954959514,2023-11-06,12:00:00,12.17,11.43,76,Rain,...,0.45,Le Clos Saint-Gilles,9.5,412.0,48.590676,-1.466182,https://www.booking.com/hotel/fr/le-clos-saint...,"Set in Ardevon, Le Clos Saint-Gilles is 8 km f...",5,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41075,La Rochelle,117858,46.1591126,-1.1520434,2023-11-11,09:00:00,13.12,12.39,73,Clouds,...,0.00,Les Halles 3,8.8,71.0,46.161448,-1.149133,https://www.booking.com/hotel/fr/les-halles-3....,"Offering free WiFi and city views, Les Halles ...",25,4
41076,La Rochelle,117858,46.1591126,-1.1520434,2023-11-11,09:00:00,13.12,12.39,73,Clouds,...,0.00,Hôtel Le Transatlantique - Lycée Hôtelier La R...,8.8,356.0,46.143734,-1.160263,https://www.booking.com/hotel/fr/lycee-hotelie...,"Located in La Rochelle, 2.5 km from Parc Expo ...",25,4
41077,La Rochelle,117858,46.1591126,-1.1520434,2023-11-11,09:00:00,13.12,12.39,73,Clouds,...,0.00,Centre Ville Location,8.7,225.0,46.161589,-1.145726,https://www.booking.com/hotel/fr/centre-ville-...,Featuring water sports facilities and free WiF...,27,3
41078,La Rochelle,117858,46.1591126,-1.1520434,2023-11-11,09:00:00,13.12,12.39,73,Clouds,...,0.00,joli petit nid douillet en centre ville,8.6,97.0,46.158113,-1.149802,https://www.booking.com/hotel/fr/joli-petit-ni...,"Ideally situated in the centre of La Rochelle,...",28,1


In [45]:
#   Creation of a database_avg_weather_df, merging the geo_weather and combined_hotel_df
database_avg_weather_df = geo_weather[['city',
                        'osm_id',
                        'lat',
                        'lon',
                        'temperature',
                        'feels_like',
                        'humidity',
                        'rain_pop',
                        'rain_mmh',
                        'expected_weather',
                        'rank_temp',
                        'rank_rain_inv',
                        'rank_avg']].merge(combined_hotel_df[['hotel_name',
                                                            'hotel_score',
                                                            'hotel_reviews',
                                                            'hotel_lat',
                                                            'hotel_lon',
                                                            'hotel_url',
                                                            'hotel_description',
                                                            'hotel_rank',
                                                            'hotel_rank_inv',
                                                            'osm_id']], on= 'osm_id')

In [46]:
#   Show the first 5 rows of the database_avg_weather_df
database_avg_weather_df

,city,osm_id,lat,lon,temperature,feels_like,humidity,rain_pop,rain_mmh,expected_weather,...,rank_avg,hotel_name,hotel_score,hotel_reviews,hotel_lat,hotel_lon,hotel_url,hotel_description,hotel_rank,hotel_rank_inv
0,Aigues&Mortes,272287,43.566152,4.191540,16.38,15.37,49.72,0.00,0.00,Clouds,...,1.5,Au Cœur des Remparts,9.9,140.0,43.565401,4.192973,https://www.booking.com/hotel/fr/au-coeur-des-...,"Located in Aigues-Mortes, 24 km from Montpelli...",1,22
1,Aigues&Mortes,272287,43.566152,4.191540,16.38,15.37,49.72,0.00,0.00,Clouds,...,1.5,Mas Sainte Marie,9.8,55.0,43.582241,4.247664,https://www.booking.com/hotel/fr/mas-sainte-ma...,"Situated in Aigues-Mortes, 29 km from Montpell...",2,21
2,Aigues&Mortes,272287,43.566152,4.191540,16.38,15.37,49.72,0.00,0.00,Clouds,...,1.5,AIGUES MARINES,9.7,105.0,43.558937,4.218141,https://www.booking.com/hotel/fr/aigues-marine...,"AIGUES MARINES has garden views, free WiFi and...",3,20
3,Aigues&Mortes,272287,43.566152,4.191540,16.38,15.37,49.72,0.00,0.00,Clouds,...,1.5,Boutique Hôtel des Remparts & Spa,9.5,225.0,43.568036,4.190344,https://www.booking.com/hotel/fr/les-remparts-...,Located in a former military station dating fr...,5,17
4,Aigues&Mortes,272287,43.566152,4.191540,16.38,15.37,49.72,0.00,0.00,Clouds,...,1.5,Lou Mirèio,9.5,137.0,43.566075,4.192579,https://www.booking.com/hotel/fr/42-rue-pasteu...,Set in Aigues-Mortes and only 24 km from Montp...,5,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,Grenoble,80348,45.187560,5.735782,1.13,0.13,87.04,0.49,0.77,Rain,...,29.0,Sweet Loft Grenoble-Love Room avec jacuzzi & s...,8.3,67.0,45.186773,5.732261,https://www.booking.com/hotel/fr/sweet-loft-gr...,"Well set in the centre of Grenoble, Sweet Loft...",24,5
1023,Grenoble,80348,45.187560,5.735782,1.13,0.13,87.04,0.49,0.77,Rain,...,29.0,Nid Douillet 2 - Appartement cosy avec garage,8.3,57.0,45.186180,5.705947,https://www.booking.com/hotel/fr/appartement-c...,Located in Grenoble and only 1.1 km from Greno...,24,5
1024,Grenoble,80348,45.187560,5.735782,1.13,0.13,87.04,0.49,0.77,Rain,...,29.0,La petite Maison,8.2,116.0,45.175570,5.713582,https://www.booking.com/hotel/fr/la-petite-mai...,Set within 2 km of WTC Grenoble and 3.8 km of ...,26,2
1025,Grenoble,80348,45.187560,5.735782,1.13,0.13,87.04,0.49,0.77,Rain,...,29.0,Le Filmophile avec garage,8.2,68.0,45.187041,5.711585,https://www.booking.com/hotel/fr/le-filmophile...,Set in Grenoble and only 700 metres from Greno...,26,2


##  Output Data

### Database_weather_df
####    Columns:
- city : city from input list
- osm_id : unique id for city, from nominatim API
- lat : latitude of city location
- lon : longitude of  city location
- day : day of predicted weather
- hour : hour of predicted weather, 3h interval
- temp : predicted temperature
- feels_like : predicted percieved temperature
- humidity : predicted humidity
- weather_main : predicted weather category
- weather_desc : predicted weather subcategory
- rain_pop : probability of rain
- rain mmh : predicted rate of rain mm/hour
- hotel_name : name of hotel
- hotel_score : score of hotel from booking.com review
- hotel_reviews : number of review on hotel page
- hotel_lat : latitude of hotel location
- hotel_lon : longitude of hotel location 
- hotel_url : url towards hotel page
- hotel_description : paragraph describing hotel
- hotel_rank : rank based on hotel score
- hotel_rank_inv : reversed hotel rank
    

In [133]:
#   Save the database_weather_df as csv files
database_weather_df.to_csv('database_weather.csv')

In [ ]:
#   Creation of a session with the boto3 library
session = boto3.Session(aws_access_key_id= #Your aws_access_key_id
                        aws_secret_access_key=# Your aws_secret_access_key
                        )

#   Creation of a resource s3
s3 = session.resource("s3")

#   Creation of a bucket
bucket = s3.Bucket('Project_Kayak_bucket')

#   Load the database_weather.csv
#database_weather_df = pd.read_csv('database_weather.csv')


#   Upload the database_weather_df to the s3 bucket
bucket.put_object(Body=database_weather_df.to_csv(index=False),
                    Key = "Project_Kayak_bucket/database_avg_weather.csv")

### Database_avg_weather_df

In [61]:
#   Save the database_avg_weather_df as csv files
database_avg_weather_df.to_csv('database_avg_weather.csv')

In [ ]:
#   Load the database_avg_weather.csv
#database_avg_weather_df = pd.read_csv('database_avg_weather.csv')

#   Upload the database_avg_weather_df to the s3 bucket
bucket.put_object(Body=database_avg_weather_df.to_csv(index=False),
                    Key = "Project_Kayak_bucket/database_avg_weather.csv")